## Descritores

Para criar modelos de aprendizado de máquina, precisamos converter estruturas químicas a um formato que possa ser interpretado pelos algoritmos utilizados. Ainda que existam algoritmos capazes de interpretar estruturas moleculares representadas na forma de texto ou grafos, o uso de números (variáveis numéricas) é o método mais usado para a criação de modelos. 

Os descritores representam características das estruturas químicas. Podem ser facilmente interpretáveis, como a massa molecular relativa. Outros descritores, como logP, têm um significado físico, mas são calculados a partir da estrutura, podendo não representar corretamente o valor real dessa variável. Também podemos contar grupos químicos, como doadores de ligação de H ou átomos de flúor. Por fim, podemos calcular descritores topológicos e não facilmente interpretáveis.

Diversos módulos estão disponíveis gratuitamente para o cálculo de descritores. Nesse notebook, vamos mostrar como calcular 200 descritores para as estruturas no [conjunto de dados de Delaney (solubilidade)](https://www.kaggle.com/c/drug-solubility-challenge/data) utilizando o pacote RDKit.

Importando os módulos necessários

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

Importando o conjunto de dados no formato .csv e convertendo para um DataFrame do pandas

In [2]:
df = pd.read_csv("delaney-processed.csv")
# Mostrar as 10 primeiras linhas
df.head(10)

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1
5,benzothiazole,-2.733,2,135.191,0,2,0,12.89,-1.50,c2ccc1scnc1c2
6,"2,2,4,6,6'-PCB",-6.545,1,326.437,0,2,1,0.00,-7.32,Clc1cc(Cl)c(c(Cl)c1)c2c(Cl)cccc2Cl
7,Estradiol,-4.138,1,272.388,2,4,0,40.46,-5.03,CC12CCC3C(CCc4cc(O)ccc34)C2CCC1O
8,Dieldrin,-4.533,1,380.913,0,5,0,12.53,-6.29,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
9,Rotenone,-5.246,1,394.423,0,5,3,63.22,-4.42,COc5cc4OCC3Oc2c1CC(Oc1ccc2C(=O)C3c4cc5OC)C(C)=C


Vamos manter somente a coluna contendo os SMILES e o valor da variável resposta (*measured log solubility in mols per litre*)

In [3]:
df = df[["smiles", "measured log solubility in mols per litre"]]

# Renomeando as colunas
df.columns=["SMILES", "Solubilidade_medida"]

Calculando os 200 descritores

In [4]:
# Esse código nos permite ver todas as colunas em um DataFrame
pd.set_option('display.max_columns', None)

In [5]:
# Anotando todos os nomes dos descritores em uma lista
names = [name[0] for name in Descriptors.descList]

# Calculando os descritores com base na coluna SMILES e adicionando à lista "descs"
calc = MoleculeDescriptors.MolecularDescriptorCalculator(names)
descs = [calc.CalcDescriptors(Chem.MolFromSmiles(cpd)) for cpd in df["SMILES"]]

# Convertendo a lista a um DataFrame e adicionando uma coluna contendo a variável resposta
descriptors_df = pd.DataFrame(descs, columns=names)
descriptors_df["Solubilidade_medida"] = df["Solubilidade_medida"]

# Ver as cinco primeiras linhas
descriptors_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Solubilidade_medida
0,10.253329,-1.701605,10.253329,0.486602,0.217518,457.432,430.216,457.158411,178,0,0.188266,-0.393567,0.393567,0.188266,0.812500,1.375000,1.968750,16.732838,9.980560,2.475238,-2.425092,2.284935,-2.613926,5.216693,-0.333729,1.654937,759.662938,23.413485,16.862520,16.862520,15.277295,9.998816,9.998816,7.601218,7.601218,5.431494,5.431494,3.506930,3.506930,-1.73,1.212120e+07,24.903474,10.926356,5.251706,182.935327,54.693144,48.831731,18.68402,0.0,0.00000,0.0,0.000000,0.0,5.261892,0.0,30.331835,5.563451,0.000000,19.282985,54.693144,0.000000,5.261892,0.0,0.0,67.515751,13.213764,35.895287,0.0,6.069221,0.000000,0.000000,0.0,0.000000,110.371240,18.947452,11.331113,11.667418,30.331835,0.0,0.000000,0.0,202.32,80.729515,41.007583,0.0,0.00000,5.563451,0.000000,0.000000,30.331835,6.069221,0.000000,18.947452,21.592355,0.000000,0.000000,79.055254,0.486602,0.000000,10.315180,-16.476443,-1.139614,0.0,0.650000,32,7,12,0,2,2,1,0,1,12,7,12,7,0,2,2,3,-3.10802,102.2816,0,7,7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.77
1,11.724911,-0.145880,11.724911,0.145880,0.811283,201.225,190.137,201.078979,76,0,0.258698,-0.468799,0.468799,0.258698,1.200000,1.933333,2.533333,16.333131,10.209837,2.102137,-2.025284,2.120187,-2.112505,6.043455,0.102071,2.148162,459.484175,10.673362,8.357948,8.357948,7.270857,4.676643,4.676643,3.210611,3.210611,2.135103,2.135103,1.340444,1.340444,-2.03,4.231896e+03,9.5

In [6]:
# Opcional: salvar o DataFrame como .csv
descriptors_df.to_csv("Delaney_descriptors.csv", sep=";", index=False)